# Exploratory Data Analysis

In [264]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

## Initial insight

In [265]:
df = pd.read_csv('bet_table.csv')
df.head()

,Stakes,Odds,Available,Num_Runners,Race_Type,Going,Days,Bet_Strength,Result
0,2,2.90,41,6,FLAT,GOOD/SOFT,33,2,LOSER
1,2,7.20,1589,10,HURDLE,FIRM,19,3,WINNER
2,2,1.04,169,14,HURDLE,HEAVY,6,2,WINNER
3,2,2.26,75,9,FLAT,FAST,15,2,LOSER
4,5,1.83,248,16,HURDLE,SOFT,14,4,LOSER


In [266]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3208 entries, 0 to 3207
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Stakes        3208 non-null   int64  
 1   Odds          3208 non-null   float64
 2   Available     3208 non-null   int64  
 3   Num_Runners   3208 non-null   int64  
 4   Race_Type     3208 non-null   object 
 5   Going         3208 non-null   object 
 6   Days          3208 non-null   int64  
 7   Bet_Strength  3208 non-null   int64  
 8   Result        3208 non-null   object 
dtypes: float64(1), int64(5), object(3)
memory usage: 225.7+ KB


,Stakes,Odds,Available,Num_Runners,Days,Bet_Strength
count,3208.000000,3208.000000,3208.000000,3208.000000,3208.000000,3208.000000
mean,2.914589,29.212291,346.107544,10.910536,27.662406,2.765898
std,1.381263,110.710028,467.496642,4.483811,19.151875,1.359028
min,2.000000,1.010000,2.000000,5.000000,5.000000,1.000000
25%,2.000000,1.607500,45.000000,7.000000,15.000000,2.000000
50%,2.000000,2.470000,139.500000,10.000000,21.000000,3.000000
75%,5.000000,5.600000,437.500000,13.000000,35.000000,4.000000
max,5.000000,900.000000,2150.000000,26.000000,125.000000,5.000000


## Feature engineering
### Calculate profit row

In [267]:
def calculate_profit(row):
    if row['Result'] == 'WINNER':
        return row['Stakes'] * row['Odds']
    elif row['Result'] == 'LOSER':
        return -row['Stakes']
    else:
        return 0

df['Profit'] = df.apply(calculate_profit, axis=1)
df.head()

,Stakes,Odds,Available,Num_Runners,Race_Type,Going,Days,Bet_Strength,Result,Profit
0,2,2.90,41,6,FLAT,GOOD/SOFT,33,2,LOSER,-2.00
1,2,7.20,1589,10,HURDLE,FIRM,19,3,WINNER,14.40
2,2,1.04,169,14,HURDLE,HEAVY,6,2,WINNER,2.08
3,2,2.26,75,9,FLAT,FAST,15,2,LOSER,-2.00
4,5,1.83,248,16,HURDLE,SOFT,14,4,LOSER,-5.00


## Visual representations

In [268]:
def calculate_win_rate(df, feature):
    ave_profit = df.groupby(feature).Profit.mean().reset_index()
    win_rate = df.Result.eq('WINNER').groupby(df[feature]).mean().mul(100).reset_index()
    feature_df = ave_profit.merge(win_rate, how='inner', on=feature)
    feature_df.rename(columns={
        'Profit': 'Mean_Profit',
        'Result': 'Win_Rate'
    }, inplace=True)
    return feature_df

### Race_Type vs Win_Rate

In [283]:
race_type_df = calculate_win_rate(df, 'Race_Type')
fig = px.bar(
    x=race_type_df['Race_Type'],
    y=race_type_df['Win_Rate'],
    color=race_type_df['Win_Rate'],
    color_continuous_scale='oxy',
    title='The effect of race types on win rate',
    labels={'x': 'Race type', 'y': 'Win rate (%)'}
    )

fig.update_layout(xaxis={'categoryorder': 'total descending'}, font={'size': 24})
fig.update_coloraxes(showscale=False)
fig.write_image('../images/race-type-win-rate.png', scale=10)
fig

### Bet_Strength vs Win_Rate

In [285]:
bet_strength_df = calculate_win_rate(df, 'Bet_Strength')
fig = px.bar(
    x=bet_strength_df['Bet_Strength'],
    y=bet_strength_df['Win_Rate'],
    color=bet_strength_df['Win_Rate'],
    color_continuous_scale='oxy',
    title='The effect of bet strength on win rate',
    labels={'x': 'Bet strength', 'y': 'Win rate (%)'}
    )

fig.update_layout(font={'size': 24})

fig.update_coloraxes(showscale=False)
fig.write_image('../images/bet-strength-win-rate.png', scale=10)
fig

### Going vs Win_Rate

In [287]:
going_df = calculate_win_rate(df, 'Going')
fig = px.bar(
    x=going_df['Going'],
    y=going_df['Win_Rate'],
    color=going_df['Win_Rate'],
    color_continuous_scale='oxy',
    title='The effect of the surface condition on win rate',
    labels={'x': 'Going (surface condition)', 'y': 'Win rate (%)'}
    )

fig.update_layout(xaxis={'categoryorder': 'total descending'}, font={'size': 24})
fig.update_coloraxes(showscale=False)
fig.write_image('../images/going-win-rate.png', scale=10)
fig

### Num_Runners vs Win_Rate

In [290]:
num_runners_df = calculate_win_rate(df, 'Num_Runners')

x = list(num_runners_df.Num_Runners)
y = list(num_runners_df.Win_Rate)
z = np.polyfit(x, y, 2)
f = np.poly1d(z)
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)  # Polyfitted x values

trace1 = go.Scatter(
    x=num_runners_df['Num_Runners'],
    y=num_runners_df['Win_Rate'],
    line = dict(color='crimson', width=4)
)

trace2 = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    name='Fit',
    line = dict(color='black', width=2, dash='dash')
    )

layout = go.Layout(
    title='The effect of the number of runners on percentage of bets won',
    xaxis_title='Number of runners',
    yaxis_title='Win rate (%)',
    showlegend=False,
    font={'size': 24}
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.write_image('../images/num-racers-win-rate.png', scale=10)
fig

### Day since last rate vs Win_Rate

In [298]:
days_df = calculate_win_rate(df, 'Days')

x = list(days_df.Days)
y = list(days_df.Win_Rate)
z = np.polyfit(x, y, 1)
f = np.poly1d(z)
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)  # Polyfitted x values

trace1 = go.Scatter(
    x=days_df['Days'],
    y=days_df['Win_Rate'],
    mode='markers',
    line = dict(color='crimson', width=4)
)

trace2 = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    name='Fit',
    line = dict(color='black', width=2, dash='dash')
    )

layout = go.Layout(
    title='The effect of the number days since last race on percentage of bets won',
    xaxis_title='Days since last race',
    yaxis_title='Win rate (%)',
    showlegend=False,
    font={'size': 24}
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.write_image('../images/days-win-rate.png', scale=10)
fig

### Availability vs Win_Rate

In [297]:
days_df = calculate_win_rate(df, 'Available')

x = list(days_df.Available)
y = list(days_df.Win_Rate)
z = np.polyfit(x, y, 1)
f = np.poly1d(z)
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)  # Polyfitted x values

trace1 = go.Scatter(
    x=days_df['Available'],
    y=days_df['Win_Rate'],
    mode='markers',
    line = dict(color='crimson', width=1)
)

trace2 = go.Scatter(
    x=x_new,
    y=y_new,
    mode='lines',
    name='Fit',
    line = dict(color='black', width=2, dash='dash')
    )

layout = go.Layout(
    title='The effect of maximum bet size on percentage of bets won',
    xaxis_title= 'Maxium bet before odds change (£)',
    yaxis_title='Win rate (%)',
    showlegend=False,
    font={'size': 24}
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
fig.write_image('../images/available-win-rate.png', scale=10)
fig